<a href="https://colab.research.google.com/github/kiuugi/pandas/blob/main/07_Combine_2)_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [참고] <a href="https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">Pandas Cheat Sheet</a>

## merge

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

<img src="https://miro.medium.com/max/1400/1*ZCpo3gXuXI4KFhKivEt2ZA.png" width="600">

- merge(데이터프레임1, 데이터프레임2) : 두 데이터프레임에 동일한 이름을 가진 컬럼을 기준으로 두 데이터프레임을 합침

<img src='https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png?ezimgfmt=ng:webp/ngcb1'>

In [2]:
import pandas as pd

In [3]:
pd.merge?

In [5]:
df1 = pd.DataFrame({"x1":['A','B','C'],"x2":[1,2,3]})
df1

,x1,x2
0,A,1
1,B,2
2,C,3


In [7]:
df2 = pd.DataFrame({"x1":['A','B','D'],"x2":['T','F','T']})
df2

,x1,x2
0,A,T
1,B,F
2,D,T


In [8]:
pd.merge(df1,df2, how='left', on='x1')

,x1,x2_x,x2_y
0,A,1,T
1,B,2,F
2,C,3,NaN


In [9]:
pd.merge(df1,df2, how='right', on='x1')

,x1,x2_x,x2_y
0,A,1.0,T
1,B,2.0,F
2,D,NaN,T


In [10]:
pd.merge(df1,df2, how='inner', on='x1')

,x1,x2_x,x2_y
0,A,1,T
1,B,2,F


In [11]:
pd.merge(df1,df2, how='outer', on='x1')

,x1,x2_x,x2_y
0,A,1.0,T
1,B,2.0,F
2,C,3.0,NaN
3,D,NaN,T


* Filterling Join

In [14]:
df2.x1

0    A
1    B
2    D
Name: x1, dtype: object

In [15]:
df1.x1

0    A
1    B
2    C
Name: x1, dtype: object

In [18]:
# df1.x1 안의 값과 df2.x1 값이 일치한 경우만 가져오기(df1 내용만)
# df1[조건]
df1[df1.x1 == df2.x1]

# isin()
df1[df1.x1.isin(df2.x1)]

,x1,x2
0,A,1
1,B,2


In [22]:
# ~ 제외하고 라는 의미
# df1[~df1.x1.isin(df2.x1)]
df1[df1.x1 != df2.x1]

,x1,x2
2,C,3


* 두 번째 실습 : indicator 옵션 활용하기 + query() + drop()

In [23]:
df1

,x1,x2
0,A,1
1,B,2
2,C,3


In [25]:
df3 = pd.DataFrame({"x1":['B','C','D'],"x2":[2,3,4]})
df3

,x1,x2
0,B,2
1,C,3
2,D,4


In [28]:
pd.merge(df1,df3)

,x1,x2
0,B,2
1,C,3


In [47]:
pd.merge(df1,df3, how='outer')

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [30]:
pd.merge(df1,df3, how='outer', indicator=True)

,x1,x2,_merge
0,A,1,left_only
1,B,2,both
2,C,3,both
3,D,4,right_only


In [33]:
pd.merge(df1,df3, how='outer', indicator=True).query('_merge == "left_only"')

,x1,x2,_merge
0,A,1,left_only


In [35]:
# 줄바꿈으로 에러가 날 때 \ 로 줄바꿈을 알려줌
pd.merge(df1,df3, how='outer', indicator=True) \
.query('_merge == "left_only"').drop(columns=['_merge'])

,x1,x2
0,A,1


### [실습2]

In [53]:
df1 = pd.DataFrame({
    "id": [1, 2, 3],
    "customer_id": [1, 2, 3],
    "customer_name": [
        "Robert",
        "Peter",
        "Dave",
    ],
})
df1

,id,customer_id,customer_name
0,1,1,Robert
1,2,2,Peter
2,3,3,Dave


In [54]:
df2 = pd.DataFrame({
    "id": [1, 2, 4],
    "customer_id": [100, 200, 300],
    "order_date": [
        "2021-01-21",
        "2021-02-03",
        "2020-10-01",
    ],
})
df2

,id,customer_id,order_date
0,1,100,2021-01-21
1,2,200,2021-02-03
2,4,300,2020-10-01


In [71]:
pd.merge(df1, df2, on='id')

,customer_id_x,customer_name,customer_id_y,order_date
id,,,,
1,1,Robert,100,2021-01-21
2,2,Peter,200,2021-02-03


In [73]:
pd.merge(df1, df2 , how='outer', on='id')

,customer_id_x,customer_name,customer_id_y,order_date
id,,,,
1,1.0,Robert,100.0,2021-01-21
2,2.0,Peter,200.0,2021-02-03
3,3.0,Dave,NaN,NaN
4,NaN,NaN,300.0,2020-10-01


In [57]:
pd.merge(df1, df2 , how='left')

,id,customer_id,customer_name,order_date
0,1,1,Robert,NaN
1,2,2,Peter,NaN
2,3,3,Dave,NaN


In [66]:
pd.merge(df1, df2 , how='right')

,customer_id,customer_name,order_date
0,100,NaN,2021-01-21
1,200,NaN,2021-02-03
2,300,NaN,2020-10-01


#### 인덱스가 있다면?

In [59]:
# 데이터 프레임 안의 특정 컬럼이 있다면
df1.set_index("id", inplace=True)

In [60]:
df2.set_index("id", inplace=True)

In [61]:
pd.merge(df1,df2, left_index=True, right_index=True)

,customer_id_x,customer_name,customer_id_y,order_date
id,,,,
1,1,Robert,100,2021-01-21
2,2,Peter,200,2021-02-03


In [69]:
pd.merge(df1,df2, left_index=True, right_index=True, how='outer')

,customer_id_x,customer_name,customer_id_y,order_date
id,,,,
1,1.0,Robert,100.0,2021-01-21
2,2.0,Peter,200.0,2021-02-03
3,3.0,Dave,NaN,NaN
4,NaN,NaN,300.0,2020-10-01


In [68]:
pd.merge(df1,df2, left_index=True, right_index=True, how='left')

,customer_id_x,customer_name,customer_id_y,order_date
id,,,,
1,1,Robert,100.0,2021-01-21
2,2,Peter,200.0,2021-02-03
3,3,Dave,NaN,NaN


In [67]:
pd.merge(df1,df2, left_index=True, right_index=True, how='right')

,customer_id_x,customer_name,customer_id_y,order_date
id,,,,
1,1.0,Robert,100,2021-01-21
2,2.0,Peter,200,2021-02-03
4,NaN,NaN,300,2020-10-01
